In [2]:
import pandas as pd
import numpy as np
import time, os

import re

### 1. Consolidated files and created DF

In [3]:
#import & consolidate files
df = pd.DataFrame()
path = os.getcwd()
files = os.listdir('/project2_rawdata')

file_names = [f for f in files]
file_names

file_names.remove('.DS_Store')
file_names.remove('.ipynb_checkpoints')

for file in file_names:
    df1 = pd.read_csv(r'/project2_rawdata/'+(file))
    df = df.append(df1)

In [4]:
df.head()

,hotel_name,rating,walk_score,res_score,attraction_score,num_reviews2,num_QA,num_Tips,price_range,num_rooms
0,Hotel 81 - Elegance,3.0,['91'],['202'],['24'],212,9.0,33.0,S$69 - S$158,120
1,Value Hotel Nice,3.5,['83'],['60'],['2'],262,12.0,38.0,S$69 - S$178,63
2,The Hive Singapore Hostel,4.0,['87'],['72'],['12'],426,4.0,54.0,S$23 - S$120,86
3,Backpacker Cozy Corner GuestHouse,3.0,['100'],['601'],['85'],140,2.0,15.0,S$42 - S$80,4
4,Fragrance Hotel - Oasis,3.5,['81'],['61'],['7'],358,27.0,53.0,S$69 - S$117,36


### 2. Removed duplicates and NULL values

In [7]:
df.sort_values("hotel_name", inplace = True) 

In [9]:
#removing duplicate hotels
df.drop_duplicates(subset ="hotel_name", keep = 'last', inplace=True)

In [11]:
#dropping NULL for the 3 scores and ratings
df.dropna(subset=['walk_score'], inplace=True)
df.dropna(subset=['rating'], inplace=True)

### 3. Cleaning cells/formatting

In [13]:
#removing characters in cells
df.walk_score.replace('\'|\'','', regex=True, inplace=True)
df.walk_score.replace('\[|\]','', regex=True, inplace=True)
df.res_score.replace('\'|\'','', regex=True, inplace=True)
df.res_score.replace('\[|\]','', regex=True, inplace=True)
df.attraction_score.replace('\'|\'','', regex=True, inplace=True)
df.attraction_score.replace('\[|\]','', regex=True, inplace=True)

df.num_reviews2.replace('\,|','', regex=True, inplace=True)
df.num_rooms.replace('\,|','', regex=True, inplace=True)

In [14]:
#formatting
df.walk_score = df.walk_score.astype(int)
df.res_score = df.res_score.astype(int)
df.attraction_score = df.attraction_score.astype(int)
df.num_reviews2 = df.num_reviews2.astype(int)
df.num_QA = df.num_QA.astype(int)
df.num_Tips = df.num_Tips.astype(int)

### 4. Removing & filling non-numeric values in num_rooms 

In [16]:
#impute median of room nums
non_num = df[pd.to_numeric(df['num_rooms'], errors='coerce').isnull()]
yes_num = df[pd.to_numeric(df['num_rooms'], errors='coerce').notnull()]

ser = pd.Series(df['num_rooms']) 
pd.to_numeric(ser, errors='coerce')

df['num_rooms'] = pd.to_numeric(ser, errors='coerce')
#filling NaNs with median 
df.num_rooms.fillna(yes_num.num_rooms.median(), inplace=True)

In [18]:
#converting to int
df.num_rooms = df.num_rooms.astype(int)

### 5.Cleaning & splitting price_range column

In [20]:
#removing $ and -
edited_price_range = df.price_range.replace('\-|\$', '', regex=True, inplace=True)
edited_price_range 

In [22]:
#removing the first S
df['price_range'] = df.price_range.str.strip('S')

In [24]:
#splitting into two columns
ser1 = pd.Series(df['price_range']) 
new = df["price_range"].str.split("S", n =1 , expand = True)
new
df["min_price"]= new[0]  
df["max_price"]= new[1] 

In [25]:
#formatting
df.max_price.replace('\,|','', regex=True, inplace=True)

In [28]:
#all rows with non-numeric values 
df[df.max_price.isnull()]

,hotel_name,rating,walk_score,res_score,attraction_score,num_reviews2,num_QA,num_Tips,price_range,num_rooms,min_price,max_price
4,Ali's Nest Singapore,4.0,98,290,17,47,4,4,ali`s nest singapore hotel singapore,10,ali`s nest singapore hotel singapore,None
21,GAS 81,3.0,69,96,6,4,0,0,gas 81 hotel singapore,186,gas 81 hotel singapore,None
11,Hostel One66,4.5,61,73,2,11,0,0,hostel one66 hotel singapore,1,hostel one66 hotel singapore,None
13,Island Resort,2.0,51,25,5,13,0,0,"island resort singapore, island hotel singapore",197,"island resort singapore, island hotel singapore",None
0,Le Imperial Hotel,3.0,100,313,27,15,0,4,le imperial hotel singapore hotel singapore,25,le imperial hotel singapore hotel singapore,None
6,Lotus at Jervois,4.5,60,159,10,17,0,4,lotus at jervois hotel singapore,20,lotus at jervois hotel singapore,None
5,Lotus at Joo Chiat,4.5,89,80,9,22,0,5,lotus at joo chiat hotel singapore,23,lotus at joo chiat hotel singapore,None
7,Raffles Town Club,4.0,49,18,2,32,1,4,raffles town club hotel singapore,30,raffles town club hotel singapore,None
4,Singapore International Campus,1.0,78,36,5,3,0,2,singapore international campus hotel singapore,180,singapore international campus hotel singapore,None
8,Singapore Mariners' Club,4.0,99,403,38,29,10,5,singapore mariners` club hotel singapore,46,singapore mariners` club hotel singapore,None


In [29]:
#impute median of room nums
non_num = df[pd.to_numeric(df['num_rooms'], errors='coerce').isnull()]
yes_num = df[pd.to_numeric(df['num_rooms'], errors='coerce').notnull()]

ser = pd.Series(df['num_rooms']) 
pd.to_numeric(ser, errors='coerce')

df['num_rooms'] = pd.to_numeric(ser, errors='coerce')
#filling NaNs with median 
df.num_rooms.fillna(yes_num.num_rooms.median(), inplace=True)

In [30]:
mp_yes_num = df[pd.to_numeric(df['max_price'], errors='coerce').notnull()]
mp_yes_num.max_price.median()

217.0

In [31]:
#filling nulls for min
min_non_num = df[pd.to_numeric(df['min_price'], errors='coerce').isnull()]
min_yes_num = df[pd.to_numeric(df['min_price'], errors='coerce').notnull()]
ser1 = pd.Series(df['min_price']) 
pd.to_numeric(ser1, errors='coerce')

df['min_price'] = pd.to_numeric(ser1, errors='coerce')
df.min_price.fillna(min_yes_num.min_price.median(), inplace=True)

In [32]:
#filling nulls for max
max_non_num = df[pd.to_numeric(df['max_price'], errors='coerce').isnull()]
max_yes_num = df[pd.to_numeric(df['max_price'], errors='coerce').notnull()]
ser2 = pd.Series(df['max_price']) 
pd.to_numeric(ser2, errors='coerce')

df['max_price'] = pd.to_numeric(ser2, errors='coerce')
df.max_price.fillna(max_yes_num.max_price.median(), inplace=True)

In [33]:
#converting min_price and max_price to float

df.max_price = df.max_price.astype(float)
df.min_price = df.min_price.astype(float)

### 6. dropping price_range and creating avg_price

In [36]:
df.drop(['price_range'], axis=1, inplace=True)

In [37]:
df.rename(columns={'num_reviews2': 'num_reviews'}, inplace=True)

In [38]:
#create new avg_price column
col = df.loc[: , "min_price":"max_price"]
df['avg_price'] = col.mean(axis=1)
df

,hotel_name,rating,walk_score,res_score,attraction_score,num_reviews,num_QA,num_Tips,num_rooms,min_price,max_price,avg_price
19,30 Bencoolen,4.5,100,352,58,545,53,58,131,186.0,379.0,282.5
6,5footway.inn Project Ann Siang,3.5,100,941,129,235,15,28,26,31.0,158.0,94.5
44,7 Wonders Hostel,3.5,97,302,18,133,9,15,9,30.0,302.0,166.0
54,7 Wonders Hostel At Upper Dickson,4.5,100,248,44,4,0,0,75,23.0,212.0,117.5
13,@ Little Red Dot,4.0,90,135,12,458,5,56,18,18.0,61.0,39.5
...,...,...,...,...,...,...,...,...,...,...,...,...
6,ibis budget Singapore Joo Chiat,3.5,97,84,10,408,34,77,90,70.0,145.0,107.5
15,ibis budget Singapore Pearl,3.5,79,66,6,367,41,36,129,67.0,129.0,98.0
29,ibis budget Singapore Ruby,3.5,80,68,4,957,50,100,168,68.0,131.0,99.5
27,ibis budget Singapore Sapphire,3.5,78,55,6,347,20,34,50,73.0,136.0,104.5


In [42]:
#PICKLE
df.to_pickle("./cleaned_data.pkl")